Main Resources:

https://github.com/opencv/opencv/issues/18120
https://stackoverflow.com/questions/9868963/cvimwrite-could-not-find-a-writer-for-the-specified-extension
https://pyimagesearch.com/2020/07/27/opencv-grabcut-foreground-segmentation-and-extraction/
https://www.geeksforgeeks.org/python-opencv-cv2-imwrite-method/

This file contains test code that runs the GrabCut algorithm on a sample image. Later it will include testing for robustness.

In [1]:
import cv2
import os
import time
import argparse

import numpy as np

In [2]:
ap = argparse.ArgumentParser()
ap.add_argument("-i", "--image", type=str,
	default=os.path.sep.join(["images", "adrian.jpg"]),
	help="path to input image that we'll apply GrabCut to")
ap.add_argument("-i", "--save-to", type=str,
	default="../../Media/Computed_Media/image_segmentation/",
	help="path to saved output images")
ap.add_argument("-c", "--iter", type=int, default=10,
	help="# of GrabCut iterations (larger value => slower runtime)")
args = vars(ap.parse_args())

usage: ipykernel_launcher.py [-h] [-i IMAGE] [-c ITER]
ipykernel_launcher.py: error: unrecognized arguments: -f /home/carwyn/.local/share/jupyter/runtime/kernel-4b31f259-12b7-4105-8461-ec0d7b943aee.json


SystemExit: 2

/home/carwyn/anaconda3/envs/hypertrophy/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [24]:
# For ipynb, simulate inputs
image = "../../Media/Images/original_leg_day/FLIR_20220906_102026_232-Visual.jpeg"
computed_media_folder = "../../Media/Computed_Media/image_segmentation/"
iter = 10

args = {"image":image, "iter":iter}

In [25]:
image = cv2.imread(args["image"])
mask = np.zeros(image.shape[:2], dtype="uint8")

# Print image with axes to estimate bounding box
 
cv2.imshow("Sample Image", image)
cv2.waitKey(0) # Wait for a keypress otherwise the kernel instantly crashes
cv2.destroyAllWindows()

rect = (246, 200, 700, 1330)


In [26]:
# allocate memory for two arrays that the GrabCut algorithm internally
# uses when segmenting the foreground from the background
fgModel = np.zeros((1, 65), dtype="float")
bgModel = np.zeros((1, 65), dtype="float")

# apply GrabCut using the the bounding box segmentation method
start = time.time()
(mask, bgModel, fgModel) = cv2.grabCut(image, mask, rect, bgModel,
	fgModel, iterCount=args["iter"], mode=cv2.GC_INIT_WITH_RECT)
end = time.time()
print("[INFO] applying GrabCut took {:.2f} seconds".format(end - start))

[INFO] applying GrabCut took 5.05 seconds


In [27]:
# the output mask has for possible output values, marking each pixel
# in the mask as (1) definite background, (2) definite foreground,
# (3) probable background, and (4) probable foreground
values = (
	("Definite Background", cv2.GC_BGD),
	("Probable Background", cv2.GC_PR_BGD),
	("Definite Foreground", cv2.GC_FGD),
	("Probable Foreground", cv2.GC_PR_FGD),
)
# loop over the possible GrabCut mask values
for (name, value) in values:
    # construct a mask that for the current value
    print("[INFO] showing mask for '{}'".format(name))
    valueMask = (mask == value).astype("uint8") * 255
    # display the mask so we can visualize it
    cv2.imshow(name, valueMask)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

[INFO] showing mask for 'Definite Background'
[INFO] showing mask for 'Probable Background'
[INFO] showing mask for 'Definite Foreground'
[INFO] showing mask for 'Probable Foreground'


In [28]:
# we'll set all definite background and probable background pixels
# to 0 while definite foreground and probable foreground pixels are
# set to 1
outputMask = np.where((mask == cv2.GC_BGD) | (mask == cv2.GC_PR_BGD),
	0, 1)
# scale the mask from the range [0, 1] to [0, 255]
outputMask = (outputMask * 255).astype("uint8")
# apply a bitwise AND to the image using our mask generated by
# GrabCut to generate our final output image
output = cv2.bitwise_and(image, image, mask=outputMask)

In [29]:
cv2.imshow("Input", image)
cv2.imshow("GrabCut Mask", outputMask)
cv2.imshow("GrabCut Output", output)
cv2.waitKey(0)
cv2.destroyAllWindows()

Write Files to Folder

In [32]:
# Save the images to Computed Media folder
cv2.imwrite(computed_media_folder + "input_image.png", image)
cv2.imwrite(computed_media_folder + "output_mask.png", outputMask)
cv2.imwrite(computed_media_folder + "output_image.png", output)

for (name, value) in values:
    valueMask = (mask == value).astype("uint8") * 255
    cv2.imwrite(computed_media_folder + name.replace(" ", '-') + ".png", valueMask)

Now that a proof of concept segmenter is in place, the next steps are as follows:

We must test the robustness of the GrabCut algorithm
 - Since this implementation of GrabCut uses an estimated bounding box of the object, this must be evaluated from the standpoint of an application for consumers. Is it realistic that a client would highlight the leg/arm/appendige in every image they want analyzed? And how accurate should it be for GrabCut to work consistently well.
 - If it is not robust, RINDNet or some other segmenter must be put in place to either improve or replace solely GrabCut.
 - The final algorithm is constrained to be something that can run on a cell phone processor or API calls from phone (low network usage).